In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os

In [36]:
driver = webdriver.Chrome('c:/chromedriver.exe')
url = 'file:///C:/Users/Playdata/Desktop/shinhancard_20201110_900046432/shinhancard_20201110_900046432.html'
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html,'html.parser')
tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr > td')

tables

[<td>20.09.27</td>,
 <td>체크<br/>549</td>,
 <td class="textL">서호김밥</td>,
 <td class="textR">            14,000</td>,
 <td class="textR">                 0</td>,
 <td class="textR">                 0</td>,
 <td class="textR">            14,000</td>,
 <td class="textR">                 0</td>,
 <td class="textR">                 0</td>,
 <td class="textR"> </td>,
 <td>20.09.27</td>,
 <td>체크<br/>549</td>,
 <td class="textL">GS25성수포레점</td>,
 <td class="textR">            10,000</td>,
 <td class="textR">                 0</td>,
 <td class="textR">                 0</td>,
 <td class="textR">            10,000</td>,
 <td class="textR">                 0</td>,
 <td class="textR">               500</td>,
 <td class="textR"> </td>,
 <td>20.09.28</td>,
 <td>체크<br/>549</td>,
 <td class="textL">왕창할인마트</td>,
 <td class="textR">             5,700</td>,
 <td class="textR">                 0</td>,
 <td class="textR">                 0</td>,
 <td class="textR">             5,700</td>,
 <td class="textR">

In [99]:
date_list = []

# for n in range(0,340,10) :
#     date = tables[n]
#     date_list.append(date.text)
    
for n in range(0,20,10) :
    date = tables[n]
    date_list.append(date.text)
    
date_list

['20.09.27', '20.09.27']

In [101]:
store_list = []

# for n in range(2,340,10) :
#     store = tables[n]
#     store_list.append(store.text)
    
for n in range(2,20,10) :
    store = tables[n]
    store_list.append(store.text)

store_list

['서호김밥', 'GS25성수포레점']

In [102]:
location_list = []

def create_url(search_text):
    headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}

    base = "https://dapi.kakao.com/v2/local/search/keyword.json"
    param_query = "?query=" +search_text
    print(f'base + param_query: {base + param_query}')
    respond = requests.get(base + param_query,headers=headers)
    return respond.json()

for n in range(len(store_list)) :
    store_name = store_list[n]
    result = create_url(store_name)
#     print(result)
    tmp1 = result['documents']
    tmp2 = tmp1[0]
    tmp3 = tmp2['address_name']
    location_list.append(tmp3)
    
location_list

base + param_query: https://dapi.kakao.com/v2/local/search/keyword.json?query=서호김밥
base + param_query: https://dapi.kakao.com/v2/local/search/keyword.json?query=GS25성수포레점


['서울 서초구 방배동 790-13', '서울 성동구 성수동1가 685-696']

In [103]:
price_list = []

# for n in range(3,340,10) :
#     price = tables[n]
#     price_list.append(price.text)
    
for n in range(3,20,10) :
    price = tables[n]
    price_list.append(price.text)
    
price_list

['            14,000', '            10,000']

In [106]:
table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }

df1 = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])

df1

,사용날짜,사용처,사용처주소,사용금액
0,20.09.27,서호김밥,서울 서초구 방배동 790-13,"14,000"
1,20.09.27,GS25성수포레점,서울 성동구 성수동1가 685-696,"10,000"


In [107]:
excel_writer = pd.ExcelWriter('c:/myPyCode/data/carduselist.xlsx', 
                              engine='xlsxwriter')
df1.to_excel(excel_writer, index = False, sheet_name='카드사용리스트')
excel_writer.save()